# Visualizing trajectories
---

### Import required packages

In [ ]:
import os
import sys
import numpy
import pandas
import sqlite3

sys.path.insert(1, "../")

from lmtanalysis import Measure
from lmtanalysis import Features

from lmtanalysis.Animal   import AnimalPool
from scripts.Set_Genotype import set_genotype
from lmtanalysis.Event    import EventTimeLine
from lmtanalysis.FileUtil import getFilesToProcess
from lmtanalysis.Util     import getAllEvents, mute_prints

from matplotlib import pyplot as plt, patches

In [ ]:
get_ipython().magic('reload_ext autoreload')
get_ipython().magic('autoreload 2')
get_ipython().magic('matplotlib qt5')

### Load a SQLite database

In [ ]:
files = getFilesToProcess()

if files and len(files) > 0:
    SQLITE_FN = files[0]
    out_dir  = os.path.dirname(SQLITE_FN)
    out_base = os.path.splitext(SQLITE_FN)[0]
else: print("No file selected...")

### Connect to data base and load detections for first 10 minutes

In [ ]:
connection = sqlite3.connect( SQLITE_FN )
animalPool = AnimalPool()

with mute_prints():
    # load infos about the animals
    animalPool.loadAnimals( connection )
    # load all detection (positions) of all animals for the first hour
    animalPool.loadDetection(
                             start = 0, 
                             end   = 10*Measure.oneMinute
                            )
    # retrieve all event names
    all_events = getAllEvents(connection=connection)
    
detections = animalPool.getDetectionTable()
print("Done")

### Plot trajectories as lines

In [ ]:
f, ax = plt.subplots(1, len(animalPool.getAnimalList()), sharex=True, sharey=True, figsize=(16,8), dpi=100)
for i, (key, df) in enumerate(detections.groupby("RFID")):
    ax[i].plot(df.x_cm, df.y_cm, color=next(ax[0]._get_lines.prop_cycler)["color"])
    ax[i].set_aspect(1.)
    arena = patches.Rectangle([0,0], 50, 50, fill=False, color="gray", linewidth=3, label="Arena")
    center_area = patches.Rectangle([Measure.CENTER_MARGIN,]*2, 50-2*Measure.CENTER_MARGIN, 50-2*Measure.CENTER_MARGIN, fill=False, color="gray", linewidth=2, linestyle=":", label="Center region")
    ax[i].add_patch(arena)
    ax[i].add_patch(center_area)
    ax[i].set_title(f"{key} ({df.genotype.iloc[0]})")
    ax[i].set_xlabel("cm")
    ax[i].set_ylabel("cm")
ax[0].invert_yaxis()

### Plot trajectories as scatter plot

In [ ]:
f, ax = plt.subplots(1, len(animalPool.getAnimalList()), sharex=True, sharey=True, figsize=(16,8), dpi=100)
for i, (key, df) in enumerate(detections.groupby("RFID")):
    ax[i].scatter(df.x_cm, df.y_cm, color=next(ax[0]._get_lines.prop_cycler)["color"], alpha=0.05)
    ax[i].set_aspect(1.)
    arena = patches.Rectangle([0,0], 50, 50, fill=False, color="gray", linewidth=3, label="Arena")
    center_area = patches.Rectangle([Measure.CENTER_MARGIN,]*2, 50-2*Measure.CENTER_MARGIN, 50-2*Measure.CENTER_MARGIN, fill=False, color="gray", linewidth=2, linestyle=":", label="Center region")
    ax[i].add_patch(arena)
    ax[i].add_patch(center_area)
    ax[i].set_title(key)
    ax[i].set_xlabel("cm")
    ax[i].set_ylabel("cm")
ax[0].invert_yaxis()